# Data Glacier Data Science:: Bank Marketing (Campaign) -- Group Project
# Bank Marketing (Campaign)
 

Problem Statement:

ABC Bank wants to sell it's term deposit product to customers and before launching the product they want to develop a model which help them in understanding whether a particular customer will buy their product or not (based on customer's past interaction with bank or other Financial Institution).

 

Why ML Model: Bank wants to use ML model to shortlist customer whose chances of buying the product is more so that their marketing channel (tele marketing, SMS/email marketing etc)  can focus only to those customers whose chances of buying the product is more.

This will save resource and their time ( which is directly involved in the cost ( resource billing)).

Develop model with Duration and without duration feature and report the performance of the model.

Duration feature is not recommended as this will be difficult to explain the result to business and also it will

be difficult for business to campaign based on duration.

In [1]:
# Import the libraries
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd

from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Importing Data

In [2]:
baf_df = pd.read_csv('/content/bank-additional-full.csv',sep=';')

In [3]:
baf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [4]:
baf_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# Data Cleaning

In [5]:
baf_df.isna().any()

age               False
job               False
marital           False
education         False
default           False
housing           False
loan              False
contact           False
month             False
day_of_week       False
duration          False
campaign          False
pdays             False
previous          False
poutcome          False
emp.var.rate      False
cons.price.idx    False
cons.conf.idx     False
euribor3m         False
nr.employed       False
y                 False
dtype: bool

In [6]:
print( f"No. of duplicated rows: {baf_df.duplicated().sum()}")

No. of duplicated rows: 12


In [7]:
baf_df = baf_df.drop_duplicates()
baf_df.shape

(41176, 21)

# Model Building

## Separating Independent and outcome varables

In [8]:
data_X = baf_df.iloc[:,:-1]
data_y = baf_df["y"]

In [9]:
print(f"Shape of X: {data_X.shape} \nShape of y: {data_y.shape}")

Shape of X: (41176, 20) 
Shape of y: (41176,)


In [10]:
data_y.replace({"no":0,"yes":1},inplace=True)

In [11]:
data_y.head()

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

## Splitting the data

We will be splitting out dataset into train and test datasets before doing any encoding or feature engineering to avoid data leakage problem.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_X,data_y,test_size=0.2)

In [13]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (32940, 20)
X_test: (8236, 20)
y_train: (32940,)
y_test: (8236,)


## One hot encoding

In [14]:
categorical_cols = data_X.columns[data_X.dtypes==object].tolist()
categorical_cols

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

def addOneHotEncoding(temp,df,vectorizer,name):
  for i, col in enumerate(vectorizer.get_feature_names_out()):
    col_name = name+"_"+col
    df[col_name] = temp[:,i].toarray().ravel().tolist()
  return df

def OneHotEncoder(categorical_cols,X_train,X_test):
  for i in categorical_cols:
    Vectorizer = CountVectorizer(token_pattern="[A-Za-z0-9-.]+")
    print("Encoding for feature: ",i)
    temp_col = Vectorizer.fit_transform(X_train[i])
    X_train = addOneHotEncoding(temp_col, X_train,Vectorizer,i)

    temp_col = Vectorizer.transform(X_test[i])
    X_test = addOneHotEncoding(temp_col, X_test, Vectorizer, i)

In [16]:
OneHotEncoder(categorical_cols, X_train, X_test)

Encoding for feature:  job
Encoding for feature:  marital
Encoding for feature:  education
Encoding for feature:  default
Encoding for feature:  housing
Encoding for feature:  loan
Encoding for feature:  contact
Encoding for feature:  month
Encoding for feature:  day_of_week
Encoding for feature:  poutcome


In [17]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

X_train: (32940, 73)
X_test: (8236, 73)


In [18]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32940 entries, 16024 to 2604
Data columns (total 73 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            32940 non-null  int64  
 1   job                            32940 non-null  object 
 2   marital                        32940 non-null  object 
 3   education                      32940 non-null  object 
 4   default                        32940 non-null  object 
 5   housing                        32940 non-null  object 
 6   loan                           32940 non-null  object 
 7   contact                        32940 non-null  object 
 8   month                          32940 non-null  object 
 9   day_of_week                    32940 non-null  object 
 10  duration                       32940 non-null  int64  
 11  campaign                       32940 non-null  int64  
 12  pdays                          32940 non-nu

In [19]:
X_train = X_train.drop(categorical_cols, axis=1)
X_test = X_test.drop(categorical_cols, axis=1)

In [20]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

X_train: (32940, 63)
X_test: (8236, 63)


In [21]:
#from sklearn.preprocessing import OrdinalEncoder
#oe=OrdinalEncoder()
#X_train[categorical_cols]=oe.fit_transform(X_train[categorical_cols])
#X_CV[categorical_cols] = oe.fit_transform(X_CV[categorical_cols])
#X_test[categorical_cols] = oe.fit_transform(X_test[categorical_cols])

## Feature Scaling

In [22]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train2 = pd.DataFrame(sc_X.fit_transform(X_train))
X_test2 = pd.DataFrame(sc_X.fit_transform(X_test))

X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values
X_train2.index = X_train.index.values
X_test2.index = X_test.index.values

X_train = X_train2
X_test = X_test2

## Model Building

###Logistic Regression

In [23]:
# with "duration" column

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

model = LogisticRegression(random_state = 42, class_weight='balanced', penalty='l2')
model.fit(X_train, y_train)
y_pred = model.predict (X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc= roc_auc_score(y_test, y_pred)


In [24]:
results = pd.DataFrame([["Logistic Regression (with Duration)",accuracy,precision,recall, f1, roc_auc]],
                       columns = ['Model','Accuracy','Precision','Recall','f1 score','ROC_AUC'])
results                       

,Model,Accuracy,Precision,Recall,f1 score,ROC_AUC
0,Logistic Regression (with Duration),0.858791,0.422931,0.890661,0.573524,0.872824


###SVM (linear)

In [25]:
from sklearn.svm import SVC
model = SVC(random_state=42, kernel = 'linear', class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict (X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc= roc_auc_score(y_test, y_pred)

In [26]:
model_result = pd.DataFrame([["SVC (linear)",accuracy,precision,recall, f1, roc_auc]],
                       columns = ['Model','Accuracy','Precision','Recall','f1 score','ROC_AUC'])
results = results.append(model_result, ignore_index=True)
results 

,Model,Accuracy,Precision,Recall,f1 score,ROC_AUC
0,Logistic Regression (with Duration),0.858791,0.422931,0.890661,0.573524,0.872824
1,SVC (linear),0.844706,0.399599,0.908884,0.555130,0.872966


###SVM (rbf)

In [27]:
from sklearn.svm import SVC
model = SVC(random_state=42,class_weight='balanced', kernel = 'rbf')
model.fit(X_train, y_train)
y_pred = model.predict (X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc= roc_auc_score(y_test, y_pred)

In [28]:
model_result = pd.DataFrame([["SVC (rbf)",accuracy,precision,recall, f1, roc_auc]],
                       columns = ['Model','Accuracy','Precision','Recall','f1 score','ROC_AUC'])
results = results.append(model_result, ignore_index=True)
results 

,Model,Accuracy,Precision,Recall,f1 score,ROC_AUC
0,Logistic Regression (with Duration),0.858791,0.422931,0.890661,0.573524,0.872824
1,SVC (linear),0.844706,0.399599,0.908884,0.555130,0.872966
2,SVC (rbf),0.856848,0.419216,0.889522,0.569865,0.871235


### Random forest

In [29]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42, n_estimators=200, criterion='entropy', class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict (X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc= roc_auc_score(y_test, y_pred)

In [30]:
model_result = pd.DataFrame([["Random Forest (n=200)",accuracy,precision,recall, f1, roc_auc]],
                       columns = ['Model','Accuracy','Precision','Recall','f1 score','ROC_AUC'])
results = results.append(model_result, ignore_index=True)
results 

,Model,Accuracy,Precision,Recall,f1 score,ROC_AUC
0,Logistic Regression (with Duration),0.858791,0.422931,0.890661,0.573524,0.872824
1,SVC (linear),0.844706,0.399599,0.908884,0.555130,0.872966
2,SVC (rbf),0.856848,0.419216,0.889522,0.569865,0.871235
3,Random Forest (n=200),0.914886,0.661202,0.413440,0.508760,0.694081


In [31]:
# K-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator= model, X= X_train, y=y_train, cv=10)

print("Random forest classifer accuracy: %0.2f" %(accuracies.mean()))

Random forest classifer accuracy: 0.91


### XGBoost

In [32]:
from sklearn.utils import class_weight
from xgboost import XGBClassifier

model = XGBClassifier(random_state=42, class_weight='balanced', max_depth=10, learning_rate=0.2)
model.fit(X_train, y_train)
y_pred = model.predict (X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc= roc_auc_score(y_test, y_pred)

In [33]:
model_result = pd.DataFrame([["XGBoost",accuracy,precision,recall, f1, roc_auc]],
                       columns = ['Model','Accuracy','Precision','Recall','f1 score','ROC_AUC'])
results = results.append(model_result, ignore_index=True)
results 

,Model,Accuracy,Precision,Recall,f1 score,ROC_AUC
0,Logistic Regression (with Duration),0.858791,0.422931,0.890661,0.573524,0.872824
1,SVC (linear),0.844706,0.399599,0.908884,0.555130,0.872966
2,SVC (rbf),0.856848,0.419216,0.889522,0.569865,0.871235
3,Random Forest (n=200),0.914886,0.661202,0.413440,0.508760,0.694081
4,XGBoost,0.915372,0.614702,0.552392,0.581884,0.755538


In [34]:
# Without "duration" feature
X_train = X_train.drop("duration", axis=1)
X_test = X_test.drop("duration", axis=1)

In [35]:
y_testmodel = LogisticRegression(random_state = 0, class_weight='balanced', penalty='l2')
model.fit(X_train, y_train)
y_pred = model.predict (X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc= roc_auc_score(y_test, y_pred)

In [36]:
model_result = pd.DataFrame([["Logistic Regression (without Duration)",accuracy,precision,recall, f1, roc_auc]],
                       columns = ['Model','Accuracy','Precision','Recall','f1 score', 'ROC_AUC'])
results = results.append(model_result, ignore_index=True)
results                       

,Model,Accuracy,Precision,Recall,f1 score,ROC_AUC
0,Logistic Regression (with Duration),0.858791,0.422931,0.890661,0.573524,0.872824
1,SVC (linear),0.844706,0.399599,0.908884,0.555130,0.872966
2,SVC (rbf),0.856848,0.419216,0.889522,0.569865,0.871235
3,Random Forest (n=200),0.914886,0.661202,0.413440,0.508760,0.694081
4,XGBoost,0.915372,0.614702,0.552392,0.581884,0.755538
5,Logistic Regression (without Duration),0.898494,0.553571,0.247153,0.341732,0.611684


In [38]:
# Without "duration" feature and class balancing

model = LogisticRegression(random_state = 0, penalty='l2')
model.fit(X_train, y_train)
y_pred = model.predict (X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

roc_auc= roc_auc_score(y_test, y_pred)

In [39]:
model_result = pd.DataFrame([["Logistic Regression (without Duration & class balancing)",accuracy,precision,recall, f1, roc_auc]],
                       columns = ['Model','Accuracy','Precision','Recall','f1 score','ROC_AUC'])
results = results.append(model_result, ignore_index=True)
results  

,Model,Accuracy,Precision,Recall,f1 score,ROC_AUC
0,Logistic Regression (with Duration),0.858791,0.422931,0.890661,0.573524,0.872824
1,SVC (linear),0.844706,0.399599,0.908884,0.555130,0.872966
2,SVC (rbf),0.856848,0.419216,0.889522,0.569865,0.871235
3,Random Forest (n=200),0.914886,0.661202,0.413440,0.508760,0.694081
4,XGBoost,0.915372,0.614702,0.552392,0.581884,0.755538
5,Logistic Regression (without Duration),0.898494,0.553571,0.247153,0.341732,0.611684
6,Logistic Regression (without Duration & class ...,0.903473,0.625378,0.235763,0.342432,0.609455
